In [ ]:
'''
When working with ecoinvent 3.8, I need to use brightway2 + a downgrade of bw2io in order to properly create the biosphere3 database and to properly load ecoinvent 3.8 cutoff

The ecoinvent 3.8 cutoff dataset has been loaded in the previous notebook, so now we need to use a different environment:
The environment setup that can do this is bw2premise-ei38compat-ok

In this notebook, we'll be using brightway2.5
'''

# Libraries

In [1]:
# Import BW25 packages.
import bw2data as bd

# 1. Support Functions

In [2]:
from database_setup import scenario_db_name

# 2. Imports and Declarations

In [3]:
bd.projects

Brightway2 projects manager with 19 objects:
	LNV-EI38
	LNV-EI38-20241204
	LNV-EI38-202502025
	LNV-EI38-20250225
	LNV-EI38-20250226
	LNV-EI38-20250227
	LNV-EI38-20250303
	LNV-EI38-20250314
	LNV-EI38-20250317
	LNV-EI38-test20241120
	LNV-EI38-test20241120-BaselineEI38
	V1_pLCA_LiIon
	V1_pLCA_liIon
	V2_pLCA_LiIon
	V2_pLCA_liIon
	V3_pLCA_liion
	default
	test-EI38-SSP1-Base
	v3_liion
Use `projects.report()` to get a report on all projects.

In [4]:
#Creating/accessing the project
# bd.projects.set_current("LNV-EI38-20250227") #Creating/accessing the project


# Fixing NMVOC:
# bd.projects.set_current("LNV-EI38-20250303") #Creating/accessing the project

# Fixing logistic regression:
bd.projects.set_current("LNV-EI38-20250317") #Creating/accessing the project

# 3. Incorporating Background Scenarios with Premise

In [5]:
# From here:
# https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios
from premise import *

In [ ]:
'''
We'll incorporate the following scenarios, each per database:
TRUE - SSP1-None (REMIND SSP1-Base)
- SSP1-RCP1.9 (REMIND SSP1-PkBudg500)
- SSP1-RCP2.6 (REMIND SSP1-PkBudg1150)
TRUE - SSP2-None (REMIND SSP2-Base)
- SSP2-RCP1.9 (REMIND SSP2-PkBudg500)
- SSP2-RCP2.6 (REMIND SSP2-PkBudg1150)
TRUE - SSP5-Base

From:
2025 to 2040

And Create:
- Baseline (keep without transforming)
- BL + VSI
'''

## 2.1. Defining the scenarios

In [6]:
# I want to work in between 2025 and 2040, therefore:
years = range(2025, 2041, 5)

In [7]:
# The scenarios defined previously are:
scenario_definitions = [
    {"model": "remind", "pathway": "SSP1-Base"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP1-PkBudg500"}, 
    # {"model": "remind", "pathway": "SSP1-PkBudg1150"},
    {"model": "remind", "pathway": "SSP2-Base"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP2-PkBudg500"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP2-PkBudg1150"}, # Confirmed working
    {"model": "remind", "pathway": "SSP5-Base"}, # Confirmed working
]

In [8]:
# Generating my list:
scenarios = [
    {**scenario, "year": year}
    for scenario in scenario_definitions
    for year in years
]

# List of custom database names with a suffix
suffix_to_include = 'baseline'
db_names = [scenario_db_name(scenario["model"], scenario["pathway"], scenario["year"], suffix=suffix_to_include) for scenario in scenarios]

In [9]:
scenarios

[{'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2040},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2040},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2040}]

# 4. Creating baseline databases

In [10]:
# Creating the new databases:
ndb = NewDatabase(
      scenarios=scenarios,
      source_db="ecoinvent 3.8 cutoff",
      source_version="3.8", 
      key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
      use_multiprocessing=True,
      keep_uncertainty_data=False
  )

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [11]:
'''
Up to this point, we'll have the new databases (defined by the scenarios) added to the project.

We still need to transform the database based on the updates we want to run.

Some update examples here:
https://github.com/polca/premise/blob/master/examples/examples.ipynb

List of ndb.update() methods:
https://github.com/polca/premise/blob/3bb7a9c9bc0b72165450f27c3dce1a2cbedd523e/premise/new_database.py#L866

'''

"\nUp to this point, we'll have the new databases (defined by the scenarios) added to the project.\n\nWe still need to transform the database based on the updates we want to run.\n\nSome update examples here:\nhttps://github.com/polca/premise/blob/master/examples/examples.ipynb\n\nList of ndb.update() methods:\nhttps://github.com/polca/premise/blob/3bb7a9c9bc0b72165450f27c3dce1a2cbedd523e/premise/new_database.py#L866\n\n"

In [12]:
ndb.update() # <- updates eveyrthing!

## More info here:
# https://premise.readthedocs.io/en/latest/transform.html

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external:  90%|████████████▌ | 9/10 [13:51<01:32, 92.43s/it]


External scenarios (datapackages) are not provided. Skipped.
Done!



In [13]:
db_names

['EI38_cutoff_remind_SSP1-Base_2025_baseline',
 'EI38_cutoff_remind_SSP1-Base_2030_baseline',
 'EI38_cutoff_remind_SSP1-Base_2035_baseline',
 'EI38_cutoff_remind_SSP1-Base_2040_baseline',
 'EI38_cutoff_remind_SSP2-Base_2025_baseline',
 'EI38_cutoff_remind_SSP2-Base_2030_baseline',
 'EI38_cutoff_remind_SSP2-Base_2035_baseline',
 'EI38_cutoff_remind_SSP2-Base_2040_baseline',
 'EI38_cutoff_remind_SSP5-Base_2025_baseline',
 'EI38_cutoff_remind_SSP5-Base_2030_baseline',
 'EI38_cutoff_remind_SSP5-Base_2035_baseline',
 'EI38_cutoff_remind_SSP5-Base_2040_baseline']

In [14]:
# Writing the databases to Brightway 
## Considering this to be the baseline
ndb.write_db_to_brightway(name=db_names)

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 10:25:48
  Finished: 03/17/2025 10:26:31
  Total time elapsed: 00:00:42
  CPU %: 28.50
  Memory %: 34.25
Created database: EI38_cutoff_remind_SSP1-Base_2025_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 10:29:06
  Finished: 03/17/2025 10:29:49
  Total time elapsed: 00:00:43
  CPU %: 34.10
  Memory %: 34.39
Created database: EI38_cutoff_remind_SSP1-Base_2030_baseline


...

Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 10:58:17
  Finished: 03/17/2025 10:59:03
  Total time elapsed: 00:00:46
  CPU %: 23.40
  Memory %: 34.73
Created database: EI38_cutoff_remind_SSP5-Base_2030_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 11:02:17
  Finished: 03/17/2025 11:03:01
  Total time elapsed: 00:00:43
  CPU %: 16.30
  Memory %: 34.76
Created database: EI38_cutoff_remind_SSP5-Base_2035_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 11:06:25
  Finished: 03/17/2025 11:07:12
  Total time elapsed: 00:00:46
  CPU %: 34.20
  Memory %: 34.81
Created database: EI38_cutoff_remind_SSP5-Base_2040_baseline
Generate scenario report.
Report saved under c:\Users\berna\OneDrive\Documentos\GitHub\plca-NMC-oxide-BM-PhD\export\scenario_report.
Generate change report.
Report saved under c:\Users\berna\OneDrive\Documentos\GitHub\plca-NMC-oxide-BM-PhD.


# 5. Also creating the VSI anciliary database

In [15]:
# List of custom database names with a suffix
suffix_to_include = 'VSI'
db_names = [scenario_db_name(scenario["model"], scenario["pathway"], scenario["year"], suffix=suffix_to_include) for scenario in scenarios]

In [16]:
db_names

['EI38_cutoff_remind_SSP1-Base_2025_VSI',
 'EI38_cutoff_remind_SSP1-Base_2030_VSI',
 'EI38_cutoff_remind_SSP1-Base_2035_VSI',
 'EI38_cutoff_remind_SSP1-Base_2040_VSI',
 'EI38_cutoff_remind_SSP2-Base_2025_VSI',
 'EI38_cutoff_remind_SSP2-Base_2030_VSI',
 'EI38_cutoff_remind_SSP2-Base_2035_VSI',
 'EI38_cutoff_remind_SSP2-Base_2040_VSI',
 'EI38_cutoff_remind_SSP5-Base_2025_VSI',
 'EI38_cutoff_remind_SSP5-Base_2030_VSI',
 'EI38_cutoff_remind_SSP5-Base_2035_VSI',
 'EI38_cutoff_remind_SSP5-Base_2040_VSI']

In [17]:
# Creating the new databases:
ndb = NewDatabase(
      scenarios=scenarios,
      source_db="ecoinvent 3.8 cutoff",
      source_version="3.8", 
      key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
      use_multiprocessing=True,
      keep_uncertainty_data=False
  )

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [18]:
ndb.update() # <- updates eveyrthing!

## More info here:
# https://premise.readthedocs.io/en/latest/transform.html

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external:  90%|████████████▌ | 9/10 [14:19<01:35, 95.46s/it]


External scenarios (datapackages) are not provided. Skipped.
Done!



In [19]:
# Writing the databases to Brightway 
## Considering this to be the baseline
ndb.write_db_to_brightway(name=db_names)

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 11:34:32
  Finished: 03/17/2025 11:35:18
  Total time elapsed: 00:00:46
  CPU %: 29.20
  Memory %: 47.20
Created database: EI38_cutoff_remind_SSP1-Base_2025_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 11:38:56
  Finished: 03/17/2025 11:39:40
  Total time elapsed: 00:00:43
  CPU %: 14.60
  Memory %: 47.23
Created database: EI38_cutoff_remind_SSP1-Base_2030_VSI


...

Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 12:27:07
  Finished: 03/17/2025 12:27:52
  Total time elapsed: 00:00:44
  CPU %: 14.60
  Memory %: 34.46
Created database: EI38_cutoff_remind_SSP5-Base_2030_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:47


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 12:34:36
  Finished: 03/17/2025 12:35:24
  Total time elapsed: 00:00:47
  CPU %: 29.80
  Memory %: 29.94
Created database: EI38_cutoff_remind_SSP5-Base_2035_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 03/17/2025 12:41:11
  Finished: 03/17/2025 12:41:55
  Total time elapsed: 00:00:43
  CPU %: 29.60
  Memory %: 28.34
Created database: EI38_cutoff_remind_SSP5-Base_2040_VSI
Generate scenario report.
Report saved under c:\Users\berna\OneDrive\Documentos\GitHub\plca-NMC-oxide-BM-PhD\export\scenario_report.
Generate change report.
Report saved under c:\Users\berna\OneDrive\Documentos\GitHub\plca-NMC-oxide-BM-PhD.


In [20]:
list(bd.databases)

['biosphere3',
 'ecoinvent 3.8 cutoff',
 'EI38_cutoff_remind_SSP1-Base_2025_baseline',
 'EI38_cutoff_remind_SSP1-Base_2030_baseline',
 'EI38_cutoff_remind_SSP1-Base_2035_baseline',
 'EI38_cutoff_remind_SSP1-Base_2040_baseline',
 'EI38_cutoff_remind_SSP2-Base_2025_baseline',
 'EI38_cutoff_remind_SSP2-Base_2030_baseline',
 'EI38_cutoff_remind_SSP2-Base_2035_baseline',
 'EI38_cutoff_remind_SSP2-Base_2040_baseline',
 'EI38_cutoff_remind_SSP5-Base_2025_baseline',
 'EI38_cutoff_remind_SSP5-Base_2030_baseline',
 'EI38_cutoff_remind_SSP5-Base_2035_baseline',
 'EI38_cutoff_remind_SSP5-Base_2040_baseline',
 'EI38_cutoff_remind_SSP1-Base_2025_VSI',
 'EI38_cutoff_remind_SSP1-Base_2030_VSI',
 'EI38_cutoff_remind_SSP1-Base_2035_VSI',
 'EI38_cutoff_remind_SSP1-Base_2040_VSI',
 'EI38_cutoff_remind_SSP2-Base_2025_VSI',
 'EI38_cutoff_remind_SSP2-Base_2030_VSI',
 'EI38_cutoff_remind_SSP2-Base_2035_VSI',
 'EI38_cutoff_remind_SSP2-Base_2040_VSI',
 'EI38_cutoff_remind_SSP5-Base_2025_VSI',
 'EI38_cutoff_remi